[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/012-Seabin.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

! python3 -m pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@6331a1e36d8e3cdb4cfbc3539f31bdad1eb465a7" 


O(m)=||d-Fm||+e^2||Am|

O(p) || F P p||



In [ ]:
from sep_python import default_io, Hypercube, FloatVector
import numpy as np

vec=default_io.vector_from_storage("../data/seabin.HH")


In [ ]:
from giee import BoxFilter, ConvOpAdjFilter, convOpAdjData

In [ ]:
import numba
from generic_solver._pyOperator import Operator

In [ ]:
from sep_plot import Grey

known=vec.clone()
unknown=vec.clone()
@numba.njit()
def known_data(vec,known,unknown):
    for i2 in range(vec.shape[0]):
        for i1 in range(vec.shape[1]):
            if vec[i2,i1]==0:
                known[i2,i1]=0
                unknown[i2,i1]=1
            else:
                known[i2,i1]=1
                unknown[i2,i1]=0
known_data(vec.get_nd_array(),
           known.get_nd_array(),unknown.get_nd_array())
Grey(known)

In [ ]:
from generic_solver  import ProblemL2Linear,ProblemL2LinearReg, LCGsolver, BasicStopper 
from generic_solver._pyOperator import DiagonalOp
from giee import Lap2D
j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=Lap2D(model,data)
eps=100

problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
#solve_base.setDefaults(save_model=True)
solve_base.run(problemLap1)

Grey(problemLap1.model)



In [ ]:
from sep_python import get_sep_vector
from numba import njit
@njit()
def covFunc(model,nm1,nm2,cov):
    for i2 in range(model.shape[0]):
        for i1 in range(model.shape[1]):
            for im2 in range(-nm2,nm2):
                if i2+im2 >=0 and i2+im2 < model.shape[0]:
                    for im1 in range(-nm1,nm1):
                        if i1+im1 >=0 and i1+im1 < model.shape[1]:
                            cov[im2+nm2,im1+nm1]+=(model[i2,i1]*model[i2+im2,i1+im1])
cov=get_sep_vector(ns=[81,81],os=[-40,-40],ds=[1,1])
hyper=Hypercube.set_with_ns([81,81],os=[-40,-40],ds=[1,1])
covFunc(vec.getNdArray(),40,40,cov.get_nd_array())
Grey(cov,pclip=100)

In [ ]:
from generic_solver._pyOperator import DiagonalOp, ChainOperator
from sep_python import Hypercube

def findPef(data,known,sh,zero):
    filt=BoxFilter.PEF(sh,zero)
    op=ConvOpAdjFilter(filt,data,data)
    wt=filt.create_mask(known)
    wtOp=DiagonalOp(wt)
    # 
    #  W(d-Lm) = Wd -WLm
    #
    duse=data.clone()
    data.scale(-1.)
    wtOp.forward(False,data,duse)
    wt_pef=ChainOperator(op,wtOp)

    prob=ProblemL2Linear(filt,data,wt_pef)
    #-d = D f  
    stop=BasicStopper(niter=1000)
    solve=LCGsolver(stop)
    solve.run(prob)
    return prob.model,prob.res,wt


In [ ]:
pef,res,wt=findPef(vec,known,(4,7),(0,3))
Grey(res)+Grey(vec)


In [ ]:
j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=convOpAdjData(model,data,pef)
eps=50

problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg)
solve_base=LCGsolver(problemStop)
solve_base.run(problemLap1)

Grey(problemLap1.model,pclip=99)+Grey(vec)

In [ ]:
res_reg=problemLap1.res.vecs[1]
res_reg.multiply(wt)
tot=np.sum(np.abs(res_reg.get_nd_array()))
nzero=np.sum(wt.get_nd_array())
avg=tot/nzero/eps
rnd=res_reg.clone()
rnd.rand()
rnd.scale(avg*1.8)

In [ ]:
j_op=DiagonalOp(unknown)
problemStop=BasicStopper(niter=1000)
data=vec.clone()
model=data.clone()
reg=convOpAdjData(model,data,pef)
eps=50

problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg,prior_model=rnd)
solve_base=LCGsolver(problemStop)
solve_base.run(problemLap1)

Grey(problemLap1.model,pclip=99)+Grey(vec)

In [ ]:
rnd.rand()
rnd.scale(avg*2.5)
problemLap1=ProblemL2LinearReg(model,data,j_op,eps,reg_op=reg,prior_model=rnd)
solve_base=LCGsolver(problemStop)
solve_base.run(problemLap1)

Grey(problemLap1.model,pclip=99)+Grey(vec)